# 3.2 Градиентный спуск

В этом задании нам предстоит реализовать классический алгоритм градиентного спуска для обучения модели логистической регрессии.

Алгоритм выполнения этого задания следующий:

* На основе посчитанных в первом задании частных производных, напишем функцию подсчета градиента бинарной кросс-энтропии по параметрам модели

* Напишем функцию обновления весов по посчитанным градиентам

* Напишем функцию тренировки модели

Замечание:
Тренировка модели проводится в несколько циклов, в рамках каждого из которых мы обновим веса модели, основываясь на предсказании для **каждого** объекта из датасета. Такие циклы называются *эпохами*. То есть одна эпоха - это набор обновлений весов, реализованный согласно посчитанным для каждого объекта из датасета ошибкам модели.

Вам необходимо реализовать обучение модели в несколько эпох. Их количество задается параметром функции. В рамках каждой эпохи необходимо пройти циклом по всем объектам обучающей выборки и обновить веса модели.

In [6]:
import numpy as np

np.random.seed(42)
# Функция подсчета градиента
def gradient(y_true: int, y_pred: float, x: np.array) -> np.array:
    """
    y_true - истинное значение ответа для объекта x
    y_pred - значение степени принадлежности объекта x классу 1, предсказанное нашей моделью
    x - вектор признакового описания данного объекта

    На выходе ожидается получить вектор частных производных H по параметрам модели, предсказавшей значение y_pred
    Обратите внимание, что размерность этого градиента должна получиться на единицу больше размерности x засчет своободного коэффициента a0
    """
    grad = x*((1-y_true)*y_pred - y_true*(1-y_pred))
    return grad


# Функция обновления весов
def update(alpha: np.array, gradient: np.array, lr: float):
    """
    alpha: текущее приближения вектора параметров модели
    gradient: посчитанный градиент по параметрам модели
    lr: learning rate, множитель перед градиентом в формуле обновления параметров
    """
    alpha_new = alpha - lr*gradient
    return alpha_new


# функция тренировки модели
def train(
    alpha0: np.array, x_train: np.array, y_train: np.array, lr: float, num_epoch: int
):
    """
    alpha0 - начальное приближение параметров модели
    x_train - матрица объект-признак обучающей выборки
    y_train - верные ответы для обучающей выборки
    lr - learning rate, множитель перед градиентом в формуле обновления параметров
    num_epoch - количество эпох обучения, то есть полных 'проходов' через весь датасет
    """
    alpha = alpha0.copy()
    for epo in range(num_epoch):
        for i, x in enumerate(x_train):
            x = np.hstack([x,np.array([1])]) # Добавил в конец строки признаков объекта единицу чтобы (она домнажается на свободный член альфа0)
            y_pred = 1/(1+np.exp(alpha*x))
            grad = gradient(y_train[i],y_pred,x)
            alpha = update(alpha,grad,lr)
    return alpha

# Замечания:

1. В случае, если у Вас возникли сложности с выполнением первого задания и, как следствие, у Вас не выходит сделать это, мы рекомендуем подробно ознакомиться с главой **Производные $\frac{\partial H}{\partial \omega_i}$** нашей [лекции](https://colab.research.google.com/drive/1xjX_YnXcRr8HSiYLByMHxEIAADqs7QES?usp=sharing).

2. Обращайте внимание на названия и порядок аргументов в сдаваемых на проверку функциях - они должны совпадать с тем, что указано в шаблоне кода.

3. Обратите внимание, что матрица объект-признак в описании параметров функций обозначает переменную типа numpy.array(), каждый элемент которой - объект типа numpy.array() - вектор признаков соответствующего объекта.

4. Считайте, что свободный коэффициент a0 находится **в конце** списка alpha.

Сигмоид $$σ(x) = \frac{1}{1 + e^{-x}}$$

В задаче бинарной классификации в качестве loss-функции обычно используется бинарная кросс-энтропия (Binary Cross-Entropy, BCE). Эта функция зависит от настоящих меток $y$ и предсказаний алгоритма $a(x) = p$, и выглядит следующим образом:

$$H(p, y) = - (y \cdot ln(p) +(1 - y) \cdot ln(1-p))$$

Из математического анализа известно, что если мы имеем функцию $F = F(x_1 ... x_n)$, то вектор [частных производных](https://colab.research.google.com/drive/1BODfwl4F3c7h0CE-t88zavkZWyNe8n5G#scrollTo=aGVowH1ufYP_) этой функции (называемый градиентом), $$\vec{grad(F)} = \vec{∇F} = (\frac{\partial F(x_1)}{\partial x_1} ... \frac{\partial F(x_n)}{\partial x_n})$$ 
направлен в сторону **наискорейшего роста функции**. Вектор же **антиградиента**, соответственно, направлен в направлении наискорейшего убывания функции. Если мы хотим *шаг за шагом* приближаться к минимуму функции, мы должны каждый раз делать небольшой шажок в направлении антиградиента. Математически эту идею можно записать в виде следующей формулы: $$\vec{x_{n+1}} = \vec{x_n} - λ\vec{∇F(x)}$$ Где $λ$ характеризует размер нашего шага.

Применив эту идею для поиска минимума Loss-функции по параметрам нашей модели $\vec{w}$, мы найдем оптимальный (или почти оптимальный) вектор параметров модели, который и будем использовать при классификации. Запомним эту идею!

 Производные $\frac{\partial H}{\partial \omega_i}$

В рамках алгоритма градиентного спуска нам необходимо посчитать [производные](https://colab.research.google.com/drive/1BODfwl4F3c7h0CE-t88zavkZWyNe8n5G#scrollTo=1m66zhVnUv9w) функции бинарной кросс-энтропии по параметрам модели, чтобы делать шаги градиентного спуска.

Покажем, как можно посчитать эту производную.

Рассмотрим случай, когда нам необходимо посчитать производные всего для одного примера. То есть мы имеем единственный объект $x$ с истинной меткой $y$ и гипотезой нашего алгоритма $p$. Тогда $$H(p, y) = - (y \cdot ln(p) +(1 - y) \cdot ln(1-p))$$

Причем $p = σ(ω_1 \cdot x_1 + ... + ω_{n} \cdot x_n + ω_0 ⋅ 1)$

Вспомним простое правило из математического анализа о вычисленнии [производной сложной функции](https://colab.research.google.com/drive/1BODfwl4F3c7h0CE-t88zavkZWyNe8n5G#scrollTo=qny-9SFZUibG):

$$\frac{∂ H}{∂ ω_i} = \frac{∂ H}{∂ p} \cdot \frac{∂ p}{∂ ω_i}$$
Посчитаем первое и второе слагаемое по отдельности.
- $\frac{∂ H}{∂ p} = -y ⋅ \frac{1}{p} + (1-y) \cdot \frac{1}{1 - p}$

Вспомним одно из основных свойств сигмоидальной функции (его доказательство - полезное упражнение, рекомендуется выполнить его самостоятельно):
$σ' = σ \cdot (1 - σ)$
- $\frac{∂ p}{∂ ω_i} = p⋅(1 - p)⋅x_i$, если $i \neq 0$
- $\frac{∂ p}{∂ ω_0} = p⋅(1 - p)$

Введём обозначение $x_0 ≡ 1$

Тогда оба равенства можно записать $\frac{∂ p}{∂ ω_i} = p⋅(1 - p)⋅x_i$

Перемножим полученные результаты:

$$\frac{∂ H}{∂ ω_i} = p⋅(1 - p)⋅x_i ⋅ (-y ⋅ \frac{1}{p} + (1-y) \cdot \frac{1}{1 - p}) = x_i((1 - y)⋅p - y⋅(1-p))$$